In [2]:
import xml.etree.ElementTree as ET

In [16]:
tree = ET.parse('./data/mondial_database_less.xml')

In [19]:
root = tree.getroot()
root.tag

'mondial'

In [21]:
for child in root:
    print(child.tag, child.attrib)

country {'car_code': 'AL', 'capital': 'cty-Albania-Tirane', 'memberships': 'org-BSEC org-CEI org-CD org-SELEC org-CE org-EAPC org-EBRD org-EITI org-FAO org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICCt org-Interpol org-IDA org-IFRCS org-IFC org-IFAD org-ILO org-IMO org-IMF org-IOC org-IOM org-ISO org-OIF org-ITU org-ITUC org-IDB org-MIGA org-NATO org-OSCE org-OPCW org-OAS org-OIC org-PCA org-UN org-UNCTAD org-UNESCO org-UNIDO org-UPU org-WCO org-WFTU org-WHO org-WIPO org-WMO org-UNWTO org-WTO', 'area': '28750'}
country {'car_code': 'GR', 'capital': 'cty-Greece-Athens', 'memberships': 'org-AG org-BIS org-BSEC org-CD org-SELEC org-CE org-EMU org-EAPC org-EBRD org-ECB org-EIB org-CERN org-ESA org-EU org-FATF org-FAO org-IGAD org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICJ org-ICCt org-Interpol org-IDA org-IEA org-IFRCS org-IFC org-IFAD org-IHO org-ILO org-IMO org-IMSO org-IMF org-IOC org-IOM org-OIF org-ITSO org-ITU org-ITUC org-MIGA org-NATO org-NEA org-NSG org-OECD org-OSCE org-OPC

In [47]:
root[1][0].text

'Greece'

In [67]:
data = {'country':[],'infant_mortality': []}
for element in root.iterfind('country'):
    #get name and infant mortality of all countries
    country = element.find('name')
    mortality = element.find('infant_mortality')
    data['country'].append(country.text)
        
    if mortality is not None:
        data['infant_mortality'].append(mortality.text)
    else:
        data['infant_mortality'].append('')
        
print(data)

{'infant_mortality': ['13.19', '4.78', '7.9', '6.16', '', '', '3.69'], 'country': ['Albania', 'Greece', 'Macedonia', 'Serbia', 'Montenegro', 'Kosovo', 'Andorra']}


In [69]:
import pandas as pd
import numpy as np

In [106]:
# create a pandas dataframe from the data
df = pd.DataFrame(data)

# clean up countries with missing infant_mortality rates
df = df.replace('',np.NaN)
df[['infant_mortality']] = df[['infant_mortality']].astype(float)
df.dtypes

# find the 10 lowest infant mortality rates
df = df.dropna().sort_values('infant_mortality')
df

,country,infant_mortality
6,Andorra,3.69
1,Greece,4.78
3,Serbia,6.16
2,Macedonia,7.90
0,Albania,13.19


In [124]:
data = {'country':[],'city': [], 'year':[], 'population':[]}

for city in root.findall('country/city'):
    #print(city.find('name').text)
    for element in list(city):
        if element.tag == 'name':
            print(element.text)
        elif element.tag == 'population':
            population = element.text
            year = element.attrib['year']
            print(population,year)

Tirana
Tirane
192000 1987
244153 1990
418495 2011
Shkodër
62000 1987
77075 2011
Durrës
60000 1987
113249 2011
Vlorë
56000 1987
79513 2011
Elbasan
53000 1987
78703 2011
Korçë
52000 1987
51152 2011
Skopje
506926 2002
514967 2011
Kumanovo
105484 2002
107745 2011
Beograd
1407073 1987
1576124 2002
1639121 2011
Novi Sad
299294 2002
335701 2011
Niš
250518 2002
257867 2011
Podgorica
136473 2003
150977 2011
Prishtine
Pristina
148090 1981
199654 1991
198214 2011
Andorra la Vella
15600 1987
20787 2001
22256 2011


In [126]:
# 2. 10 cities with the largest population
cityname = ""
citypopulation = ""
document = ET.parse( './data/mondial_database_less.xml' )
df = pd.DataFrame(columns=['CityName','Population']) #create data frame to hold country name and its popuplation
df['Population'] = df['Population'].astype(float)

#loop through country element to find city name and its population
for country in document.iterfind( 'country' ):
    for city in country.iter('city'): #find all cities within each country element
        cityname = city.find('name').text
        year = int(0)
        for node in city.iterfind('population'): #find all population elements with each city
            year = node.attrib['year'] #there are multiple population elements with different 'year' attribute
            if node.attrib['year'] >= year: #store the population number of the latest year
                citypopulation = int(node.text)
        df.loc[len(df)] = [cityname,citypopulation] #add city name and its population to data frame
        cityname = ""
        

#sort data frame to find 10 cities with largest population
df.sort_values(by = 'Population', ascending=False).head(10)

,CityName,Population
30,Beograd,1639121.0
8,Athina,664046.0
28,Skopje,514967.0
0,Tirana,418495.0
31,Novi Sad,335701.0
16,Thessaloniki,325182.0
32,Niš,257867.0
12,Patra,213984.0
13,Kozani,213984.0
34,Prishtine,198214.0


In [127]:
# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
countryname = ""
countrypopulation = 0
ethnicpopulation = 0
document = ET.parse( './data/mondial_database_less.xml' )
df = pd.DataFrame(columns=['Country','EthnicGroup','Population'])
df['Population'] = df['Population'].astype(float)

#loop through country element to find ethnic groups and its population
for country in document.iterfind( 'country' ):
    countryname = country.find('name').text #find country name
    year = int(0)
    countrycpopulation = int(0)
    for node in country.iterfind('population'): #find population of the country
        year = node.attrib['year']
        if node.attrib['year'] >= year: #find population of the latest year
            countrypopulation = (node.text) 
    ethnicname = None
    ethnicpopulation = 0
    for ethnic in country.iter('ethnicgroup'): #find all ethnic groups within the same country
        ethnicname = ethnic.text
        #compute each ethnic population: country population * ethnic group percentage
        ethnicpopulation = round(float(ethnic.attrib['percentage']) * 0.01 * int(countrypopulation))
        if ethnicname == None:
            ethnicname = countryname
            ethnicpopulation = countrypopulation
        df.loc[len(df)] = [countryname,ethnicname,ethnicpopulation] #store ethnic group population to data frame
    countryname = ""

#group ethnic group across all countries and sum them up to find top 10 ethnic groups and its total population
df.groupby('EthnicGroup').sum().sort_values(by = 'Population', ascending=False).head(10)

,Population
EthnicGroup,
Greek,10143150.0
Serb,6138517.0
Albanian,4805362.0
Macedonian,1322388.0
Montenegrin,330698.0
Hungarian,277706.0
Bosniak,177774.0
Roma,99689.0
Serbian,86694.0


In [136]:
document = ET.parse( './data/mondial_database.xml' )
# create a lookup table for country codes
car_codes = {'country':[], 'code':[]}

# extract country info from xml
for country in document.findall('country'):
    name = ''
    code = ''
    for node in list(country):
        if node.tag == 'name':           
            car_codes['country'].append(node.text)
            car_codes['code'].append(country.attrib['car_code'])

# create a dataframe for the country codes
# will be used for merge later
countries = pd.DataFrame(car_codes)
countries.head(10)

,code,country
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro
5,KOS,Kosovo
6,AND,Andorra
7,F,France
8,E,Spain
9,A,Austria


In [137]:
# create a table for river information
river_data = {'car_code':[], 'name':[], 'length':[]}

# extract river info from xml
for river in document.findall('river'):
    
    code = river.attrib['country'] # country code, not country name
    name = ''
    length = 0
    for node in list(river):
        if node.tag == 'name':           
            name = node.text
        elif node.tag == 'length':
            length = float(node.text)
            
    if name and code and length:
        # only add rivers with known lengths
        codes = code.split(' ') # some rivers cross multiple countries
        for c in codes:
            river_data['car_code'].append(c)
            river_data['name'].append(name) # river name
            river_data['length'].append(length)

# create a dataframe for the rivers
rivers = pd.DataFrame(river_data)
rivers.head(10)

,car_code,length,name
0,IS,230.0,Thjorsa
1,IS,206.0,Joekulsa a Fjoellum
2,N,604.0,Glomma
3,N,322.0,Lagen
4,S,93.0,Goetaaelv
5,N,460.0,Klaraelv
6,S,460.0,Klaraelv
7,S,470.0,Umeaelv
8,S,520.0,Dalaelv
9,S,320.0,Vaesterdalaelv


In [138]:
# create a table for lake information
lake_data = {'car_code':[], 'name':[], 'area':[]}

# extract lake info from xml
for river in document.findall('lake'):
   
    code = river.attrib['country'] # country code, not country name
    name = ''
    area = 0
    for node in list(river):
        if node.tag == 'name':           
            name = node.text
        elif node.tag == 'area':
            area = float(node.text)
            
    if name and code and area:
        # only add lakes with known areas
        codes = code.split(' ') # some lakes cross multiple countries
        for c in codes:
            lake_data['car_code'].append(c)
            lake_data['name'].append(name) # lake name
            lake_data['area'].append(area)

# create a dataframe for the lakes
lakes = pd.DataFrame(lake_data)
lakes.head(10)

,area,car_code,name
0,1040.0,SF,Inari
1,928.0,SF,Oulujaervi
2,472.0,SF,Kallavesi
3,4370.0,SF,Saimaa
4,1118.0,SF,Paeijaenne
5,368.0,N,Mjoesa-See
6,173.0,S,Storuman
7,290.0,S,Siljan
8,1140.0,S,Maelaren
9,5648.0,S,Vaenern


In [139]:

# create a table for airport information
airport_data = {'car_code':[], 'name':[], 'elevation':[]}

# extract airport info from xml
for airport in document.findall('airport'):
   
    code = airport.attrib['country'] # country code, not country name
    name = ''
    elev = 0
    for node in list(airport):
        if node.tag == 'name':           
            name = node.text
        elif node.tag == 'elevation':
            if node.text is not None:
                elev = float(node.text)
            
    if name and code and elev:
        # only add lakes with known areas
        #codes = code.split(' ') # some lakes cross multiple countries
        #for c in codes:
        airport_data['car_code'].append(code)
        airport_data['name'].append(name) # airport name
        airport_data['elevation'].append(elev)

# create a dataframe for the lakes
airports = pd.DataFrame(airport_data)
airports.head(10)

,car_code,elevation,name
0,AFG,977.0,Herat
1,AFG,1792.0,Kabul Intl
2,AL,38.0,Tirana Rinas
3,DZ,811.0,Cheikh Larbi Tebessi
4,DZ,822.0,Batna Airport
5,DZ,6.0,Soummam
6,DZ,1377.0,Tamanrasset
7,DZ,88.0,Biskra
8,DZ,691.0,Mohamed Boudiaf Intl
9,DZ,1024.0,Ain Arnat Airport


In [135]:
# merge the river and country tables
df = pd.merge(rivers, countries, how='left', left_on='car_code', right_on='code')

# find the longest river
grouped = df.groupby(['name'])[['length']].max()
longest = grouped.sort_values('length', ascending=False).ix[0]

# display info on longest river
print('Longest River:')
df[df.name == longest.name][['name','country','length']]

KeyError: 0.0

In [141]:
# merge the lake and country tables
df = pd.merge(lakes, countries, how='left', left_on='car_code', right_on='code')

# find the longest river
grouped = df.groupby(['name'])[['area']].max()
largest = grouped.sort('area', ascending=False).ix[0].name

# display info on longest river
print('Largest Lake:\n')
df[df.name == largest][['name','country','area']]

Largest Lake:



C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,name,country,area
68,Caspian Sea,Russia,386400.0
69,Caspian Sea,Azerbaijan,386400.0
70,Caspian Sea,Kazakhstan,386400.0
71,Caspian Sea,Iran,386400.0
72,Caspian Sea,Turkmenistan,386400.0


In [144]:
# merge the airport and country tables
df = pd.merge(airports, countries, how='left', left_on='car_code', right_on='code')

# find the highest airport
grouped = df.groupby(['name'])[['elevation']].max()
highest = grouped.sort('elevation', ascending=False).iloc[0].name

# display info on highest airport
airport = df[df.name == highest].iloc[0]
airport

print('Highest Airport:')
print('%s in %s, Elevation: %d m' % (airport['name'], airport.country, airport.elevation))

Highest Airport:
El Alto Intl in Bolivia, Elevation: 4063 m


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
